Notebook adapted from source: https://github.com/rcamino/pytorch-notebooks/blob/master/Train%20Torchvision%20Models%20with%20Tiny%20ImageNet-200.ipynb

# Prepare Tiny ImageNet-200

Setting up [Tiny ImageNet-200](https://tiny-imagenet.herokuapp.com/), a subset of ImageNet with 200 classes.

In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip tiny-imagenet-200.zip

In [ ]:
%matplotlib inline

from __future__ import print_function

import json
import os

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

import matplotlib.pyplot as plt

Some constants for the notebook.

In [ ]:
directory = "./tiny-imagenet-200/"
num_classes = 200

## Loading and pre-processing

First we load and pre-process the data according to the pre-trained model [documentation](http://pytorch.org/docs/master/torchvision/models.html), applying transformations using [this example](https://github.com/pytorch/examples/blob/42e5b996718797e45c46a25c55b031e6768f8440/imagenet/main.py#L89-L113).

For all data, we keep images at their original 64x64 size, and do not apply flipping or cropping. We transform them to a tensor and finally normalize them to have values between 0 and 1. The normalization parameters come from the example.

In [ ]:
# modify this depending on memory constraints
batch_size = 64

# the magic normalization parameters come from the example
transform_mean = np.array([ 0.485, 0.456, 0.406 ])
transform_std = np.array([ 0.229, 0.224, 0.225 ])

train_transform = transforms.Compose([
    # transforms.Resize(32),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = transform_mean, std = transform_std),
])

val_transform = transforms.Compose([
    # transforms.Resize(32),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = transform_mean, std = transform_std),
])

traindir = os.path.join(directory, "train")
# be careful with this set, the labels are not defined using the directory structure
valdir = os.path.join(directory, "val")

train = datasets.ImageFolder(traindir, train_transform)
val = datasets.ImageFolder(valdir, val_transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=False)

assert num_classes == len(train_loader.dataset.classes)

## Label madness

[WordNet](https://wordnet.princeton.edu/) is a large lexical database of English. ImageNet uses a subset of this database as labels for the images, and the Tiny ImageNet-200 uses an even smaller subset. The Tiny ImageNet-200 comes with a map between WordNet ids and WordNet definitions.

In [ ]:
small_labels = {}
with open(os.path.join(directory, "words.txt"), "r") as dictionary_file:
    line = dictionary_file.readline()
    while line:
        label_id, label = line.strip().split("\t")
        small_labels[label_id] = label
        line = dictionary_file.readline()

In [ ]:
small_labels.items()

The train subdirectory of Tiny ImageNet-200 has a collection of subdirectories, named using to the WordNet ids to label the images that they contain. The torchvision data loader uses the names of the subdirectories as labels, but replaces them with numeric indices when iterating the batches.

In [ ]:
os.listdir(traindir)[:5]

['n03584254', 'n02403003', 'n02056570', 'n02769748', 'n01443537']

In [ ]:
labels = {}
label_ids = {}
for label_index, label_id in enumerate(train_loader.dataset.classes):
    ### label_id is the string code, label is the english word(s), and label_index is the integer class number
    label = small_labels[label_id]
    labels[label_index] = label
    label_ids[label_id] = label_index

In [ ]:
labels.items()

In [ ]:
label_ids.items()

Another problem is that the validation directory only has one subdirectory called `images`. The labels for every image inside this subdirectory are defined in a file called `val_annotations.txt`.

In [ ]:
val_label_map = {}
with open(os.path.join(directory, "val/val_annotations.txt"), "r") as val_label_file:
    line = val_label_file.readline()
    while line:
        file_name, label_id, _, _, _, _ = line.strip().split("\t")
        val_label_map[file_name] = label_id
        line = val_label_file.readline()

In [ ]:
val_label_map.items()

Finally we update the Tiny ImageNet-200 validation set labels:

In [ ]:
val_loader.dataset.imgs[:5]

In [ ]:
for i in range(len(val_loader.dataset.imgs)):
    file_path = val_loader.dataset.imgs[i][0]

    file_name = os.path.basename(file_path)
    label_id = val_label_map[file_name]

    val_loader.dataset.imgs[i] = (file_path, label_ids[label_id])

In [ ]:
train_images = []
train_labels = []

for index, epoch in enumerate(train_loader):
    # print(index)
    images, labels = epoch
    train_images.append(images)
    train_labels.append(labels)

# Concatenate the lists of batches into a single tensor
train_images = torch.cat(train_images, dim=0)
train_labels = torch.cat(train_labels, dim=0)

val_images = []
val_labels = []

for index, epoch in enumerate(val_loader):
    # print(index)
    images, labels = epoch
    val_images.append(images)
    val_labels.append(labels)

# Concatenate the lists of batches into a single tensor
val_images = torch.cat(val_images, dim=0)
val_labels = torch.cat(val_labels, dim=0)

Split 10% of the training data for validation and use the original validation set for labeled test data

In [ ]:
trainimages.size()

torch.Size([100000, 3, 64, 64])

In [ ]:
torch.manual_seed(0)

In [ ]:
torch.manual_seed(0)
indices = torch.randperm(train_images.size()[0])

trainimages=train_images[indices]
trainlabels=train_labels[indices]

tensor([36044, 73702, 76369,  2549, 52523, 32634, 95851, 73247, 29081, 26748])


In [ ]:
valsplitimages = train_images[:10000]
valsplitlabels = train_labels[:10000]
trainsplitimages = train_images[10000:]
trainsplitlabels = train_labels[10000:]


In [ ]:
trainsplitimages.size()

torch.Size([90000, 3, 32, 32])

In [ ]:
torch.save(trainsplitimages, "./X_train.pt")
torch.save(trainsplitlabels, "./y_train.pt")
torch.save(valsplitimages, "./X_val.pt")
torch.save(valsplitlabels, "./y_val.pt")
torch.save(val_images, "./X_test.pt")
torch.save(val_labels, "./y_test.pt")